# A quick start into TESPy

## General information

[TESPy](https://tespy.readthedocs.io) is an open-source framework for the simulation of component based thermodynamic
conversion processes. With the software you can use predefined components such as a pump, compressor, heat exchanger,
turbine or valve (and many more) to build thermodynamic systems. The system is set up by connecting the components in a
generic way and then specifying respective process and component parameters.

The software then performs a steady state simulation of your system by creating and solving a system of equations in the 
so-called *equation oriented (EO)* approach, i.e. components are not calculated stepwise, but simultaniously in a generic
way. The system represents the individual topology and component and process specifications provided by you. TESPy 
accomplishes this by solving for

- mass flow,
- pressure and
- enthalpy
- (as well as fluid composition)

of every connection between two components. After solving a model, missing component and process
parameters - for example: efficiencies, temperatures, pressure losses - are determined based on this information. The
EO approach lets the modeler choose, which parameters are inputs and which parameters are results: For instance, a
compressor efficiency can be an input and the system variables are solved to meet that constraint, or it can be a result
of other inputs.

## Mini example

TESPy consists of three main building blocks:

- class `Network` as container of the simulation
- class `Component` (children of them: `Compressor`, `Valve`, ...), in which
  thermodynamic conversion processes take place
- class `Connection`, which define the topology of the `Network`
  by connecting the individual components

In this example, we will create two simple networks to show the principle of TESPy. For further tutorials and examples
we recommend looking into the online documentation.

### Modeling a Compressor

First, we are modeling a compressor, which compresses fully saturated steam of a refrigerant to a higher pressure level. The
figure below shows the abstract representation of the component. The respective table summarizes the
process parameters we are going to apply in our example.

![Compressor Heat Balance Diagramm](img/Compressor.svg)



| parameter description   | model location | model parameter | value | unit |
|:----------------------- |:-------------- |:--------------- | -----:|:---- |
| saturated gas state     | in             | `x`             |   100 | %    |
| temperature             |                | `T`             |    10 | °C   |
| mass flow               |                | `m`             |   0.1 | kg/s |
| efficiency              | compressor     | `eta_s`         |    80 | %    |
| pressure ratio          |                | `pr`            |     3 | -    |

In a fist step the topology has to be set up. For this a network has to be defined. In our python code, we import the `Network` class from `tespy.networks`. After this we define our network in just calling this function. In general the initialization of `Network()` needs no arguments but it has optional arguments. In our example we use this option to set the units of pressure und temperature to $\mathrm{bar}$ and $\mathrm{°C}$ respectively.

Additionally the components of the network have to be defined. These include the compressor of course. But as every connnecion is connecting two components, we alsop need a source of the working fluid in our simple example and a sink. Consequently we import the classes `Source`, `Compressor`and `Sink` from `tespy.components`. Initializing the components it is mandatory to include a label for the component. This is done just after initializing the network.

Last but not least, the components have to be connected. This is done by using the `Connection`class that is imported from `tespy.connections`. When initializing a connection it has to be defined where it starts and where it ends. In our example we connect the first connection to the outlet of the source and the inlet of the compressor. Connections additionally need some kind of identifier which again is the label of the component.

The final step in defining the topology is adding the connections to the network using the `add_conns` method of the network instance.

In [18]:
from tespy.networks import Network
from tespy.components import Source, Sink, Compressor
from tespy.connections import Connection


nwk = Network(p_unit="bar", T_unit="C")

so = Source("source")
cp = Compressor("compressor")
si = Sink("sink")

c1 = Connection(so, "out1", cp, "in1", label="1")
c2 = Connection(cp, "out1", si, "in1", label="2")

nwk.add_conns(c1, c2)

To make a simulation it is now necessary to specify relevant component and process parameters. We start with the values
as provided in the table above. In addition to this we define the working fluid. In our case it is the refrigerant R290.
TESPy understands a fluid as a dictionary of components and mass fractions. In our example, the only component is R290
and the mass fraction of this component is 100 %, i.e. 1.

We can see that both component and process parameters can be set using the `set_attr` method of components and connections.
The arguments of this method can be different depending on the context, e.g. which component is adressed.

After that the network can be simulated using the network's `solve` method. This method needs one argument that tells TESPy
if the calculation is a design calculation or an off-design calculation. This information is passed as one can see below.

In [19]:
c1.set_attr(fluid={"R290": 1}, T=10, x=1, m=0.1)

cp.set_attr(eta_s=0.8, pr=3)

nwk.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.86e+06   | 0 %        | 0.00e+00   | 1.81e+06   | 3.88e+05   | 0.00e+00   | 0.00e+00   
 2     | 5.91e+05   | 2 %        | 0.00e+00   | 2.33e-10   | 7.39e+05   | 0.00e+00   | 0.00e+00   
 3     | 2.33e-10   | 100 %      | 0.00e+00   | 2.33e-10   | 6.94e-12   | 0.00e+00   | 0.00e+00   
 4     | 2.33e-10   | 100 %      | 0.00e+00   | 2.33e-10   | 6.94e-12   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: inf


We can have a look at the results. An overview is provided by the `print_results` method of the `Network`.

In [20]:
nwk.print_results()


##### RESULTS (Compressor) #####
+------------+----------+----------+----------+--------+
|            |        P |    eta_s |       pr |   igva |
|------------+----------+----------+----------+--------|
| compressor | 6.37e+03 | 8.00e-01 | 3.00e+00 |    nan |
+------------+----------+----------+----------+--------+
##### RESULTS (Connection) #####
+----+-----------+-----------+-----------+-----------+
|    |         m |         p |         h |         T |
|----+-----------+-----------+-----------+-----------|
|  1 | 1.000e-01 | 6.366e+00 | 5.857e+05 | 1.000e+01 |
|  2 | 1.000e-01 | 1.910e+01 | 6.494e+05 | 6.476e+01 |
+----+-----------+-----------+-----------+-----------+


Since TESPy is working with an equation oriented solver, we can now change things up. For example, instead of providing
the efficiency of the compressor, we could provide an outlet temperature. Given that temperature, the efficiency of the
compressor will be a result of the calculation.

> With the equation oriented structure the user is not constrainted in the inputs. As long as the network is well
> determined, the solver be able to find a result. One downside of the equation oriented approach is that the a initial
> guess for all variables is required. Bad starting values often lead to the solver being unable to find a solution. For
> more information please have a look at the TESPy documentation. Here in detail information and best practices are
> provided for this topic.

Unsetting a parameter of a component or connection can be done using the `set_attr` method and passing the value `None`.
Instead of unsetting the component efficiency in our example we could as well code a new script from the beginning in
which the efficiency is not set at all. The the network's `iterinfo` attribute can to be set to `False` if no detailed
information on the steps in which the problem has been solved is needed.

In [21]:
cp.set_attr(eta_s=None)  # unset the isentropic efficiency
c2.set_attr(T=70)
nwk.set_attr(iterinfo=False)
nwk.solve("design")
nwk.print_results()


##### RESULTS (Compressor) #####
+------------+----------+----------+----------+--------+
|            |        P |    eta_s |       pr |   igva |
|------------+----------+----------+----------+--------|
| compressor | 7.64e+03 | 6.67e-01 | 3.00e+00 |    nan |
+------------+----------+----------+----------+--------+
##### RESULTS (Connection) #####
+----+-----------+-----------+-----------+-----------+
|    |         m |         p |         h |         T |
|----+-----------+-----------+-----------+-----------|
|  1 | 1.000e-01 | 6.366e+00 | 5.857e+05 | 1.000e+01 |
|  2 | 1.000e-01 | 1.910e+01 | 6.621e+05 | 7.000e+01 |
+----+-----------+-----------+-----------+-----------+


For example, we can make an invalid parameter specification by setting the mass flow at the inlet and at the outlet of
the compressor. This overdetermines the system of equations and will result in an error when trying to solve.

In [22]:
c2.set_attr(T=None, m=0.1)
nwk.solve("design")

TESPyNetworkError: You cannot specify two or more values for mass flow in the same linear branch (starting at source and ending at sink).

To both train your coding capability and in this make use of the modelling and solving services TESPy provides you can write a script from scratch that uses TESPy to determine the pressure ratio that corresponds to an outlet temperature of 300 °C at ambient inlet conditions for an air compressor with an isentropic efficiency of 90 %. This is exactly waht we did in the untis "Single Components" and "Solving Strategies". TESPy obviously does all the job needed to define the jacobian matrix and to iteratively solve the generic problem. So most likely the code is quite compact...    

### Modeling a Heat Exchanger

In the second example we are going to model a heat exchanger as shown in the flowsheet below, specifically
an evaporator using heat from ambient air to completely evaporate the working fluid R290 (Propane). The 
parameters we want to use are listed in the table below.

![Heat Exchanger Heat Balance Diagram](img/HeatExchanger.svg)



| parameter description    | model location | model parameter | value | unit |
|:-----------------------  |:-------------- |:--------------- | -----:|:---- |
| refrigerant gas state    | refrigerant in | `x`             |    25 | %    |
| refrigerant temperature  |                | `T`             |     0 | °C   |
| air mass flow            | air in         | `m`             |     1 | kg/s |
| air pressure             |                | `p`             |     1 | bar  |
| air inlet temperature    |                | `T`             |     7 | °C   |
| air outlet temperature   | air out        | `T`             |     4 | °C   |
| pressure ratio hot side  | component      | `pr1`           |     1 | -    |
| pressure ratio cold side |                | `pr2`           |     1 | -    |


Similar to the compressor example we work with a `Network` instance, this time two fluids are required, i.e. air and
R290. We create the `HeatExchanger` component and connect and parametrize it according to the flowsheet and the data
listed in the table.

In order to improve the convergence a starting value for the pressure of the refrigerant can be provided. This is 
achieved by setting the parameter `p0` at the respective connection.

In [23]:
from tespy.networks import Network
from tespy.components import Source, Sink, HeatExchanger
from tespy.connections import Connection
from CoolProp.CoolProp import PropsSI as PSI


nwk = Network(p_unit="bar", T_unit="C", iterinfo=False)

so_wf = Source("working fluid source")
si_wf = Sink("working fluid sink")
so_air = Source("air source")
si_air = Sink("air sink")
eva = HeatExchanger("evaporator")

c1 = Connection(so_air, "out1", eva, "in1", label="1")
c2 = Connection(eva, "out1", si_air, "in1", label="2")
c3 = Connection(so_wf, "out1", eva, "in2", label="3")
c4 = Connection(eva, "out2", si_wf, "in1", label="4")

nwk.add_conns(c1, c2, c3, c4)

c1.set_attr(fluid={"Air": 1}, T=7, p=1, m=1)
c2.set_attr(T=4)
c3.set_attr(fluid={"R290": 1}, T=0, x=0.25)
# specification of a pressure guess value for convergence improvement
c4.set_attr(x=1, p0=5)

eva.set_attr(pr1=1, pr2=1)

nwk.solve("design")
nwk.print_results()


##### RESULTS (HeatExchanger) #####
+------------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|            |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|------------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| evaporator | -3.02e+03 | 5.63e+02 | 5.36e+00 | 7.00e+00 | 4.00e+00 |  4.00e+00 | 1.00e+00 | 1.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 |   9.58e-01 |  4.29e-01 |  9.58e-01 |
+------------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
##### RESULTS (Connection) #####


After running the simulation we can see the heat transferred from the air to the working fluid, both mass flows, or the
temperature differences between the hot side (air) and the cold side (R290), `ttd_1`. Instead of providing a fixed temperature
value for the evaporation temperature level, we can provide a temperature difference to the air temperature level. When
the air temperature changes, adjusts the evaporation pressure/temperature of the working fluid automatically.

The example below also shows, how calculated values can be accessed after a TESPy calculation. In this case
the value of the temperature at the refrigerat inlet is reffered to by `c3.T.val`.

In [24]:
eva.set_attr(ttd_l=5)
c3.set_attr(T=None)
nwk.solve("design")
c3.T.val

-1.0000000000001137

This could also be calculated for a number of different air inlet temperatures, e.g. -5 °C, 0 °C, 5 °C 
and 10 °C, and a fixed cool down of the air of let's say 3 K. Both the inlet temperatures and the resulting
evaporation temperatures can be stored in an array and by this efficiently be used.

In [25]:
T_evaporation = []
for T_air in [-5, 0, 5, 10]:
    c1.set_attr(T=T_air)
    c2.set_attr(T=T_air - 3)
    nwk.solve("design")
    T_evaporation += [round(c3.T.val, 1)]

T_evaporation

[-13.0, -8.0, -3.0, 2.0]

As an exercise you could try to determine the air mass flow needed to evaporate a certain amount of 
refrigerant or the mass of refrigerant that can be evaporated with 1 MW of heat taken from the air.
There are obviously quite a number of relevant constellation that can be covered by this setup... 

## Learn more

TESPy relies on CoolProp to provide fluid property data for a large range of different fluids . The
online documentation of TESPy provides a large variety of examples and tutorials to learn to use the software as well as
extensive background information and code documentation:

- online documentation [https://tespy.readthedocs.io](https://tespy.readthedocs.io)
- github repository [https://github.com/oemof/tespy](https://github.com/oemof/tespy)
- user forum [https://github.com/oemof/tespy/discussions](https://github.com/oemof/tespy/discussions)
